[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/syedmujahedalih/ForskLabs_DL/blob/master/CNN_Pokemon_Classify_pretrainedVGG.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2018-10-17 12:49:25--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb [following]
--2018-10-17 12:49:26--  https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 91.189.89.229, 91.189.89.228
Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|91.189.89.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232624 (1.2M) [application/x-debian-package]
Saving to: ‘google-drive-ocamlfuse_

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
classifier = Sequential()

In [0]:
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

In [0]:
#Pooling

classifier.add(MaxPooling2D(pool_size = (2, 2)))


In [0]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [0]:
# Step 3 - Flattening
classifier.add(Flatten())

In [0]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

In [0]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the test data shouldn’t be augmented!
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('drive/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('drive/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 445 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


In [0]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 100,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 100)

Epoch 1/5
  8/100 [=>............................] - ETA: 2:35 - loss: 1.1741 - acc: 0.4609

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 256s 3s/step - loss: 0.3561 - acc: 0.8608 - val_loss: 0.2240 - val_acc: 0.9249
Epoch 2/5
 99/100 [============================>.] - ETA: 1s - loss: 0.0963 - acc: 0.9621

# **Using the pretrained model: VGG16**

In [0]:
from keras.applications import VGG16

In [5]:
conv_base = VGG16(weights='imagenet',
include_top=False,
input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 6s 0us/step


In [0]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [0]:
conv_base.trainable = False #do not train the conv-base

In [0]:
import os

In [0]:
base_dir = 'drive/dataset/'
train_dir = os.path.join(base_dir, 'training_set')

test_dir = os.path.join(base_dir, 'test_set')

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='categorical')

Found 445 images belonging to 3 classes.


In [12]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='categorical')

Found 238 images belonging to 3 classes.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [14]:
history = model.fit_generator(train_generator, steps_per_epoch=100,
                              epochs=15,
                              validation_data=test_generator,
                              validation_steps=50)

Epoch 1/15
 15/100 [===>..........................] - ETA: 20:45 - loss: 1.0093 - acc: 0.5500

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 596s 6s/step - loss: 0.7098 - acc: 0.7675 - val_loss: 0.5011 - val_acc: 0.8599
Epoch 2/15
100/100 [==============================] - 116s 1s/step - loss: 0.3874 - acc: 0.9060 - val_loss: 0.3502 - val_acc: 0.8679
Epoch 3/15
100/100 [==============================] - 121s 1s/step - loss: 0.2673 - acc: 0.9285 - val_loss: 0.3073 - val_acc: 0.8851
Epoch 4/15
100/100 [==============================] - 116s 1s/step - loss: 0.2158 - acc: 0.9375 - val_loss: 0.2656 - val_acc: 0.8992
Epoch 5/15
100/100 [==============================] - 114s 1s/step - loss: 0.1757 - acc: 0.9545 - val_loss: 0.2337 - val_acc: 0.9214
Epoch 6/15
100/100 [==============================] - 121s 1s/step - loss: 0.1492 - acc: 0.9635 - val_loss: 0.2012 - val_acc: 0.9224
Epoch 7/15
100/100 [==============================] - 117s 1s/step - loss: 0.1320 - acc: 0.9630 - val_loss: 0.1947 - val_acc: 0.9163
Epoch 8/15
100/100 [==============================] - 114s 1s/step - loss: 0.117

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
    
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=test_generator,
    validation_steps=50)

Epoch 1/30
 16/100 [===>..........................] - ETA: 1:14 - loss: 0.0629 - acc: 0.9844

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 129s 1s/step - loss: 0.0327 - acc: 0.9897 - val_loss: 0.0527 - val_acc: 0.9698
Epoch 2/30
100/100 [==============================] - 117s 1s/step - loss: 0.0226 - acc: 0.9927 - val_loss: 0.0620 - val_acc: 0.9731
Epoch 3/30
100/100 [==============================] - 122s 1s/step - loss: 0.0138 - acc: 0.9962 - val_loss: 0.0428 - val_acc: 0.9798
Epoch 4/30
100/100 [==============================] - 120s 1s/step - loss: 0.0051 - acc: 0.9978 - val_loss: 0.0781 - val_acc: 0.9724
Epoch 5/30
100/100 [==============================] - 118s 1s/step - loss: 0.0055 - acc: 0.9980 - val_loss: 0.0591 - val_acc: 0.9805
Epoch 6/30
100/100 [==============================] - 123s 1s/step - loss: 0.0041 - acc: 0.9993 - val_loss: 0.0563 - val_acc: 0.9765
Epoch 7/30
100/100 [==============================] - 118s 1s/step - loss: 0.0029 - acc: 0.9993 - val_loss: 0.0374 - val_acc: 0.9738
Epoch 8/30
100/100 [==============================] - 115s 1s/step - loss: 0.002

In [37]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('drive/dataset/single_prediction/pokemon1.png', target_size = (150, 150, 3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set = train_datagen.flow_from_directory('drive/dataset/training_set',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
training_set.class_indices

Found 445 images belonging to 3 classes.


{'bulbasaur': 0, 'charmander': 1, 'pikachu': 2}